## The pandas I/O System

In [2]:
import pandas as pd
import numpy as np
# import pyarrow as pa

#### CSV - basic reading / writing

In [3]:
df = pd.DataFrame([
    ["Paul", "McCartney", 1942],
    ["John", "Lennon", 1940],
    ["Richard", "Starkey", 1940],
    ["George", "Harrison", 1943],
], columns=["first", "last", "birth"])
df = df.convert_dtypes(dtype_backend="numpy_nullable")
df
#  numpy_nullable:It converts DataFrame columns to the best possible nullable dtypes (like Int64, boolean, string) but using NumPy’s nullable extension dtypes instead of pandas’ defaults.

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


In [4]:
import io
buf = io.StringIO()

df.to_csv(buf)
print(buf.getvalue())

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943



In [5]:
df = pd.read_csv("../data/diamonds.csv", dtype_backend="numpy_nullable", nrows=1_000)
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.2,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    1000 non-null   Float64
 1   cut      1000 non-null   string 
 2   color    1000 non-null   string 
 3   clarity  1000 non-null   string 
 4   depth    1000 non-null   Float64
 5   table    1000 non-null   Float64
 6   price    1000 non-null   Int64  
 7   x        1000 non-null   Float64
 8   y        1000 non-null   Float64
 9   z        1000 non-null   Float64
dtypes: Float64(6), Int64(1), string(3)
memory usage: 85.1 KB


In [7]:
df["price"].describe()

count       1000.0
mean       2476.54
std      839.57562
min          326.0
25%         2777.0
50%         2818.0
75%         2856.0
max         2898.0
Name: price, dtype: Float64

In [12]:
df["carat"].describe()

count      1000.0
mean      0.68928
std      0.195291
min           0.2
25%           0.7
50%          0.71
75%          0.79
max          1.27
Name: carat, dtype: Float64

In [13]:
df["cut"].unique()

<StringArray>
['Ideal', 'Premium', 'Good', 'Very Good', 'Fair']
Length: 5, dtype: string

In [14]:
df["color"].unique()

<StringArray>
['E', 'I', 'J', 'H', 'F', 'G', 'D']
Length: 7, dtype: string

In [16]:
def startswith_c(column_name: str) -> bool:
    return column_name.startswith("c")

pd.read_csv(
    "../data/diamonds.csv",
    dtype_backend="numpy_nullable",
    usecols=startswith_c, # usecols gets only the columns whose names start with "c"
)

,carat,cut,color,clarity
0,0.23,Ideal,E,SI2
1,0.21,Premium,E,SI1
2,0.23,Good,E,VS1
3,0.29,Premium,I,VS2
4,0.31,Good,J,SI2
...,...,...,...,...
53935,0.72,Ideal,D,SI1
53936,0.72,Good,D,SI1
53937,0.7,Very Good,D,SI1
53938,0.86,Premium,H,SI2


### Microsoft Excel - basic reading / writing

In [17]:
df = pd.DataFrame([
    ["Paul", "McCartney", 1942],
    ["John", "Lennon", 1940],
    ["Richard", "Starkey", 1940],
    ["George", "Harrison", 1943],
], columns=["first", "last", "birth"])
df = df.convert_dtypes(dtype_backend="numpy_nullable")
df

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


In [18]:
import io
buf = io.BytesIO()
df.to_excel(buf)
buf.seek(0)
pd.read_excel(buf, dtype_backend="numpy_nullable")

,Unnamed: 0,first,last,birth
0,0,Paul,McCartney,1942
1,1,John,Lennon,1940
2,2,Richard,Starkey,1940
3,3,George,Harrison,1943


In [19]:
buf.seek(0)
pd.read_excel(buf, dtype_backend="numpy_nullable", index_col=0)

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


### Microsoft Excel - finding tables in non-default locations

In [26]:
pd.read_excel(
    "../data/beatles.xlsx",
    dtype_backend="numpy_nullable",
    sheet_name="the_data",
    skiprows=4,
    usecols="C:E",
)

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


In [20]:
pd.read_excel(
    "../data/beatles.xlsx",
    dtype_backend="numpy_nullable",
    sheet_name="the_data",
    skiprows=4,
    usecols=["first", "last", "birth"],
)

,first,last,birth
0,Paul,McCartney,1942
1,John,Lennon,1940
2,Richard,Starkey,1940
3,George,Harrison,1943


### Microsoft Excel - hierarchical data

In [21]:
df = pd.read_excel(
    "../data/hierarchical.xlsx",
    dtype_backend="numpy_nullable",
    index_col=[0, 1],
    header=[0, 1],
)
df

Year                  2024            2025         
Quarter                 Q1      Q2      Q1       Q2
Region  Sub-Region                                 
America East             1       2       4        8
        West            16      32      64      128
        South          256     512    1024     4096
Europe  West          8192   16384   32768    65536
        East        131072  262144  524288  1048576

In [22]:
df.loc[(slice(None), "East"), (slice(None), "Q2")]

,Year,2024,2025
,Quarter,Q2,Q2
Region,Sub-Region,,
America,East,2,8
Europe,East,262144,1048576
